# Task 1 - Disease Named Entity Recognition (D-NER)
## Preprocessing Steps

### Introduction

This project will utilize the Mimic III dataset as training data for a Kndole sequence tagger model.
For the development and evaluation of the model, the NCIT dataset will be used, for which the existing tags are as follows:
DiseaseClass, SpecificDisease, Modifier, CompositeMention
In order to supplement the Mimic III dataset for this task, three other datasets where consulted to aid in the fitting of specific categories for the matching with the structure of the classes:

National Cancer Institute Thesaurus: NCI9d - Thesaurus.txt: NCI Thesaurus Version 22.09d - flattened from the owl format 
from their own webpage: https://evs.nci.nih.gov/evs-download/thesaurus-downloads

The vast ULMS dataset (Unified Medical Language System),
after authentification, to be accessed at: https://www.nlm.nih.gov/research/umls/licensedcontent/umlsknowledgesources.html
Specifically the table NB.DB

MEDIC - CTD's MEDIC disease vocabulary is a modified subset of descriptors from the “Diseases” [C] branch of the U.S. National Library of Medicine's Medical Subject Headings (MeSH®),combined with genetic disorders from the Online Mendelian Inheritance in Man® (OMIM®) database. Comparative Toxicogenomics Database: https://ctdbase.org/downloads/#alldiseases



### Examination

In the following section each of the datasets in use will be examined, the relevant sections stripped from them and further processed into lists of strings to be used in rule based labeling functions.

In [2]:
import pandas as pd
D_ICD_DIAGNOSES=pd.read_csv('../../../mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv', sep=',', header=0)
D_ICD_DIAGNOSES.head(5)


,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [3]:
DiseaseClass1=D_ICD_DIAGNOSES.loc[:,'SHORT_TITLE']
DC1=set(DiseaseClass1.values.tolist())
SpecificDisease1=D_ICD_DIAGNOSES.loc[:,'LONG_TITLE']
SD1=set(SpecificDisease1.values.tolist())

In [4]:
CTD_diseases=pd.read_csv('./CTD_diseases.csv', sep=',', header=0)
CTD_diseases.head(5)

,DiseaseName,DiseaseID,AltDiseaseIDs,Definition,ParentIDs,TreeNumbers,ParentTreeNumbers,Synonyms,SlimMappings
0,10p Deletion Syndrome (Partial),MESH:C538288,NaN,NaN,MESH:D002872|MESH:D025063,C16.131.260/C538288|C16.320.180/C538288|C23.55...,C16.131.260|C16.320.180|C23.550.210.050.500.500,"Chromosome 10, 10p- Partial|Chromosome 10, mon...",Congenital abnormality|Genetic disease (inborn...
1,13q deletion syndrome,MESH:C535484,NaN,NaN,MESH:D002872|MESH:D025063,C16.131.260/C535484|C16.320.180/C535484|C23.55...,C16.131.260|C16.320.180|C23.550.210.050.500.500,Chromosome 13q deletion|Chromosome 13q deletio...,Congenital abnormality|Genetic disease (inborn...
2,15q24 Microdeletion,MESH:C579849,DO:DOID:0060395,NaN,MESH:D002872|MESH:D008607|MESH:D025063,C10.597.606.360/C579849|C16.131.260/C579849|C1...,C10.597.606.360|C16.131.260|C16.320.180|C23.55...,15q24 Deletion|15q24 Microdeletion Syndrome|In...,Congenital abnormality|Genetic disease (inborn...
3,16p11.2 Deletion Syndrome,MESH:C579850,NaN,NaN,MESH:D001321|MESH:D002872|MESH:D008607|MESH:D0...,C10.597.606.360/C579850|C16.131.260/C579850|C1...,C10.597.606.360|C16.131.260|C16.320.180|C23.55...,NaN,Congenital abnormality|Genetic disease (inborn...
4,"17,20-Lyase Deficiency, Isolated",MESH:C567076,NaN,NaN,MESH:D000312,C12.050.351.875.253.090.500/C567076|C12.200.70...,C12.050.351.875.253.090.500|C12.200.706.316.09...,"17-Alpha-Hydroxylase-17,20-Lyase Deficiency, C...",Congenital abnormality|Endocrine system diseas...


In [7]:
DiseaseClass2=CTD_diseases.loc[:,'SlimMappings']
DC2_input= DiseaseClass2.values.tolist()
DC2=[]
for former_row in DC2_input:
    Sublist=str(former_row).split('|')
    for item in Sublist:
        DC2.append(item) 
DC2=set(DC2)

In [8]:
SpecificDisease2=CTD_diseases.loc[:,'DiseaseName']
SD2=set(SpecificDisease2.values.tolist())
SpecificDisease3=CTD_diseases.loc[:,'Synonyms']
SD3=set(SpecificDisease3.values.tolist())

In [9]:
NCI9d=pd.read_csv('./Thesaurus.text', sep='\t')
NCI9d_disease_subset=NCI9d.loc[NCI9d['Therapeutic or Preventive Procedure'] == 'Disease or Syndrome']
NCI9d_disease_subset.head()

,C100000,<http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C100000>,C99521,"Percutaneous Coronary Intervention for ST Elevation Myocardial Infarction-Stable-Over 12 Hours From Symptom Onset|PERCUTANEOUS CORONARY INTERVENTION (PCI) FOR ST ELEVATION MYOCARDIAL INFARCTION (STEMI) (STABLE, >12 HRS FROM SYMPTOM ONSET)","A percutaneous coronary intervention is necessary for a myocardial infarction that presents with ST segment elevation and the subject does not have recurrent or persistent symptoms, symptoms of heart failure or ventricular arrhythmia. The presentation is past twelve hours since onset of symptoms. (ACC)",Unnamed: 5,Unnamed: 6,Therapeutic or Preventive Procedure
12,C100012,<http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,C50797,Severe Cardiac Valve Regurgitation,Evidence of severe retrograde blood flow throu...,NaN,NaN,Disease or Syndrome
21,C100020,<http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,C35317,Three Vessel Coronary Disease|THREE VESSEL DIS...,There was greater than or equal to 50% stenosi...,NaN,NaN,Disease or Syndrome
24,C100023,<http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,C35317,Two Vessel Coronary Disease|TWO VESSEL DISEASE,There was greater than or equal to 50% stenosi...,NaN,NaN,Disease or Syndrome
67,C100062,<http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,C99938,Chronic Total Coronary Artery Occlusion,Prolonged complete obstruction of the coronary...,NaN,NaN,Disease or Syndrome
76,C100070,<http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,C35279,Coronary Venous Dissection,A tear within the wall of a coronary vein. (ACC),NaN,NaN,Disease or Syndrome


In [61]:
SpecificDisease4=NCI9d_disease_subset.iloc[:,3]
SD4=set(SpecificDisease4.values.tolist())

In [80]:
ULMS_terminology=pd.read_csv("./NC.DB", sep = '|')
Modifiers=ULMS_terminology.iloc[:,0]
Mod_pre=Modifiers.values.tolist()

In [63]:
def skip_brackets(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

    

In [85]:
Mod=[]

for term in Mod_pre:
    a=skip_brackets(str(term))
    Mod.append(a)

for term in Mod_pre:
    x=term.replace('(','').replace(')','')
    Mod.append(x)
    
Mod=set(Mod)

In [ ]:
SD1 = [str(x).lower() for x in SD1]
SD2 = [str(x).lower() for x in SD2]
SD3 = [str(x).lower() for x in SD3]
SD4 = [str(x).lower() for x in SD4]
DC1 = [str(x).lower() for x in DC1]
DC2 = [str(x).lower() for x in DC2]
Mod = [str(x).lower() for x in Mod]

For the Class of CompositeDisease the occurence of a term of DiseaseClass or SpecificDisease followed by another will be used, with an instance of and, or or / \ between them.

### Mapping